In [2]:
pip install mailbox

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
pip install chardet


   ---------------------------------------- 199.4/199.4 kB 4.0 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
import mailbox
import os
import email
import re
import chardet
from html.parser import HTMLParser

In [26]:
mbox_file = r"C:\Users\ganes\OneDrive\Desktop\Datasets\takeout-20240302T195011Z-001\Takeout\Mail\mails.mbox"

In [29]:
output_dir = r"C:\Users\ganes\OneDrive\Desktop\Datasets\gk_Emails01.txt"

In [28]:
class HTMLEmailParser(HTMLParser):
    def __init__(self):
        super().__init__()
        self.plain_text = ""
        self.recording = False

    def handle_data(self, data):
        if self.recording:
            self.plain_text += data

    def handle_starttag(self, tag, attrs):
        if tag == "body":
            self.recording = True

    def handle_endtag(self, tag):
        if tag == "body":
            self.recording = False

In [30]:
os.makedirs(output_dir, exist_ok=True)

In [13]:
mbox = mailbox.mbox(mbox_file)

In [32]:
for message in mbox:
    # Get the message body
    msg = email.message_from_bytes(message.as_bytes())
    body = ''
    if msg.is_multipart():
        for part in msg.walk():
            if part.get_content_type() == 'text/html':
                payload = part.get_payload(decode=True)
                encoding = chardet.detect(payload)['encoding']
                try:
                    html_body = payload.decode(encoding or 'utf-8', errors='replace')
                except (UnicodeDecodeError, LookupError):
                    html_body = payload.decode('ascii', errors='replace')
                parser = HTMLEmailParser()
                parser.feed(html_body)
                body += parser.plain_text
    else:
        if msg.get_content_type() == 'text/html':
            payload = msg.get_payload(decode=True)
            encoding = chardet.detect(payload)['encoding']
            try:
                html_body = payload.decode(encoding or 'utf-8', errors='replace')
            except (UnicodeDecodeError, LookupError):
                html_body = payload.decode('ascii', errors='replace')
            parser = HTMLEmailParser()
            parser.feed(html_body)
            body = parser.plain_text

    # Sanitize the Message-ID header to use as a filename
    message_id = message['Message-ID']
    if message_id:
        filename = re.sub(r'[^\w.-]', '_', message_id) + '.txt'
    else:
        filename = 'unknown.txt'

    # Create a .txt file with the message body
    filepath = os.path.join(output_dir, filename)
    with open(filepath, 'w', encoding='utf-8') as f:
        f.write(body)

In [ ]:
#code below this doesn't work 

In [24]:
# Loop through each message in the .mbox file
for message in mbox:
    # Get the message body
    msg = email.message_from_bytes(message.as_bytes())
    body = ''
    if msg.is_multipart():
        for part in msg.walk():
            if part.get_content_type() == 'text/plain':
                payload = part.get_payload(decode=True)
                encoding = chardet.detect(payload)['encoding']
                try:
                    body += payload.decode(encoding or 'utf-8', errors='replace')
                except (UnicodeDecodeError, LookupError):
                    body += payload.decode('ascii', errors='replace')
    else:
        payload = msg.get_payload(decode=True)
        encoding = chardet.detect(payload)['encoding']
        try:
            body = payload.decode(encoding or 'utf-8', errors='replace')
        except (UnicodeDecodeError, LookupError):
            body = payload.decode('ascii', errors='replace')

    # Sanitize the Message-ID header to use as a filename
    message_id = message['Message-ID']
    if message_id:
        filename = re.sub(r'[^\w.-]', '_', message_id) + '.txt'
    else:
        filename = 'unknown.txt'

    # Create a .txt file with the message body
    filepath = os.path.join(output_dir, filename)
    with open(filepath, 'w', encoding='utf-8') as f:
        f.write(body)